## Demo (Neuromeka TVE) - White board sweeping

### Prepare the task scene
* Prepare 2 Indy7, and install sweeping tool to indy. (Check release/Figs/7.1.WhiteBoardSweeping.jpg)
* Prepare "floor", "track", "box1", "box2" objects as defined in pkg.detector.aruco.marker_config.py
* The robots and "track" are installed on "floor".
* "box1" and "box2" are on "track"

## set running directory to project source

In [1]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))
from pkg.utils.code_scraps import add_indy_sweep_tool, add_indy_gripper_asm3, \
        use_current_place_point_only, use_current_sub_binders_only, \
        finish_L_shape, set_l_shape_object, ModeSwitcher, double_sweep_motions

## init combined robot config

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *
INDY_IP2 = "192.168.0.106"

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, ((0.3,-0.4,0), (0,0,np.pi/2)),
                INDY_IP),
    RobotConfig(1, RobotType.indy7, ((-0.3,-0.4,0), (0,0,np.pi/2)),
                INDY_IP2, specs={"no_sdk": True})]
              , connection_list=[False, False])

connection command:
indy0: False
indy1: False


In [3]:
from pkg.detector.aruco.marker_config import *
from pkg.detector.aruco.stereo import ArucoStereo
from pkg.detector.camera.realsense import RealSense
from pkg.detector.camera.kinect import Kinect
aruco_map = get_aruco_map()
stereo = ArucoStereo(aruco_map, [Kinect(), RealSense()])
stereo.initialize()
stereo.calibrate()

Device configuration: 
	color_format: 3 
	(0:JPG, 1:NV12, 2:YUY2, 3:BGRA32)

	color_resolution: 5 
	(0:OFF, 1:720p, 2:1080p, 3:1440p, 4:1536p, 5:2160p, 6:3072p)

	depth_mode: 3 
	(0:OFF, 1:NFOV_2X2BINNED, 2:NFOV_UNBINNED,3:WFOV_2X2BINNED, 4:WFOV_UNBINNED, 5:Passive IR)

	camera_fps: 2 
	(0:5 FPS, 1:15 FPS, 2:30 FPS)

	synchronized_images_only: False 
	(True of False). Drop images if the color and depth are not synchronized

	depth_delay_off_color_usec: 0 ms. 
	Delay between the color image and the depth image

	wired_sync_mode: 0
	(0:Standalone mode, 1:Master mode, 2:Subordinate mode)

	subordinate_delay_off_master_usec: 0 ms.
	The external synchronization timing.

	disable_streaming_indicator: False 
	(True or False). Streaming indicator automatically turns on when the color or depth camera's are in use.


Start streaming


[(array([[1.82983423e+03, 0.00000000e+00, 1.91572046e+03],
         [0.00000000e+00, 1.82983423e+03, 1.09876086e+03],
         [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
  array([ 7.09966481e-01, -2.73409390e+00,  1.45804870e-03, -3.24774766e-04,
          1.44911301e+00,  5.84310412e-01, -2.56374550e+00,  1.38472950e+00])),
 (array([[1.39560388e+03, 0.00000000e+00, 9.62751587e+02],
         [0.00000000e+00, 1.39531934e+03, 5.47687012e+02],
         [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
  array([0., 0., 0., 0., 0.])),
 array([[ 0.8298847 , -0.02273035, -0.5574717 ,  0.64667064],
        [ 0.1683687 ,  0.9627916 ,  0.21138655, -0.16007583],
        [ 0.5319242 , -0.26928726,  0.80283314,  0.08677655],
        [ 0.        ,  0.        ,  0.        ,  1.        ]],
       dtype=float32)]

## create scene builder

In [4]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(stereo)
s_builder.reset_reference_coord(ref_name="floor")

## detect robot and make geometry scene
* **[IMPORTANT]** Make sure robot is visible, or correct saved file is used

In [5]:
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
# save_pickle("data/xyz_rpy_robots.pkl", xyz_rpy_robots)
xyz_rpy_robots = load_pickle("data/xyz_rpy_robots.pkl")
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)

Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## init planning pipeline

In [6]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


```
open web ui on <your ip>:8050
click geometry items / Handles / Binders to highlight geometry on RVIZ
other functions may be buggy.. please report
```

## add environment
* **[IMPORTANT]** Make sure environment items are visible, or correct saved file is used

In [7]:
gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.ENVIRONMENT])


# add cam poles
ptems = s_builder.add_poles({"cam0": s_builder.ref_coord_inv[:3,3], "cam1":np.matmul(s_builder.ref_coord_inv, stereo.T_c12)[:3,3]},
                            color=(0.6,0.6,0.6,0.0))
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)
gscene.NAME_DICT["floor"].dims = (2,1,0.01)
gscene.set_workspace_boundary( -1, 1, -0.5, 0.5, -0.1, 1.05)

 * Debug mode: off
Please create a subscriber to the marker


In [8]:
gscene.update_markers_all()

## add environmental components

In [9]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepFramer, FixtureSlot
from pkg.planning.constraint.constraint_subject import \
        CustomObject, Grasp2Point, PlacePoint, SweepFrame, SweepTask, BoxObject, FixturePoint, AbstractObject

from pkg.planning.constraint.constraint_common import MotionConstraint
from pkg.planning.constraint.constraint_subject import AbstractTask
from pkg.planning.constraint.constraint_subject import SweepLineTask

add_indy_sweep_tool(gscene, "indy0", face_name="brush_face", tool_offset=0.115)
add_indy_gripper_asm3(gscene, "indy1")

gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip1", link_name="indy1_tcp", 
                 dims=(0.01,)*3, center=(0,0,0.12), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

pscene.create_binder(bname="grip1", gname="grip1", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="brush_face", gname="brush_face", _type=SweepFramer, point=(0,0,gscene.NAME_DICT['brush_face'].dims[2]/2), 
                     rpy=(0,np.pi,0))

track = s_builder.detect_and_register(item_names=["track"], level_mask=[DetectionLevel.ENVIRONMENT])["track"]
track_face = gscene.copy_from(track, new_name="track_face", collision=False)
track_face.dims = (track.dims[0]-0.1, track.dims[1]-0.15, track.dims[2])
pscene.create_binder(bname="track_face", gname="track_face", _type=PlacePlane, point=None)
# track.dims = (track.dims[0]-0.32, track.dims[1]-0.12, track.dims[2])

gscene.update_markers_all()

TOOL_DIM = [0.06, 0.10]
TACK_WIDTH = 0.15
TRACK_DIM = track_face.dims
TRACK_NUM = np.ceil(np.divide(TRACK_DIM[1]-TOOL_DIM[1], TOOL_DIM[1])).astype(np.int)+1
TRACK_STEP = (TRACK_DIM[1]-TOOL_DIM[1])/(TRACK_NUM-1)
WP_REF_A = -np.subtract(TRACK_DIM[:2], TOOL_DIM[:2])/2
WP_REF_B = np.array([-WP_REF_A[0], WP_REF_A[1]])
WP_REF_M = np.array([0, WP_REF_A[1]])
TRC_THIC = TRACK_DIM[2]
track_list = []
for i_trc in range(TRACK_NUM):
    wp1 = gscene.create_safe(GEOTYPE.BOX, "wp{}a".format(i_trc+1), "base_link", (TOOL_DIM[0]/2,TOOL_DIM[1]/2,TRC_THIC), 
                             tuple(WP_REF_A+[0,TRACK_STEP*i_trc])+(0,),rpy=(0,0,0), 
                             color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="track_face")
    wp2 = gscene.create_safe(GEOTYPE.BOX, "wp{}b".format(i_trc+1), "base_link", (TOOL_DIM[0]/2,TOOL_DIM[1]/2,TRC_THIC), 
                             tuple(WP_REF_B+[0,TRACK_STEP*i_trc])+(0,),rpy=(0,0,0), 
                             color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="track_face")
    face = gscene.create_safe(GEOTYPE.BOX, "face{}".format(i_trc+1), "base_link", 
                              (TRACK_DIM[0] - TOOL_DIM[0]/2,TACK_WIDTH,TRC_THIC), 
                             center=tuple(WP_REF_M+[0,TRACK_STEP*i_trc])+(1e-2,),rpy=(0,0,0), 
                             color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="track_face")
    track_list.append((wp1, wp2, face))
    
    gscene.update_markers_all()

sweep_list = []
for i_t, track_tem in enumerate(track_list):
    wp1, wp2, face = track_tem
    sweep_ = pscene.create_subject(oname="sweep{}".format(i_t+1), gname="track_face", _type=SweepLineTask, 
                                   action_points_dict = {wp1.name: SweepFrame(wp1.name, wp1, [0,0,0.005], [0,0,0]),
                                                       wp2.name: SweepFrame(wp2.name, wp2, [0,0,0.005], [0,0,0])}, 
                                   clearance=[face])
    sweep_list.append(sweep_)
    


## add movable objects

In [10]:
gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])
gscene.update_markers_all()
    
for gname in sorted(gtem_dict.keys()):
    pscene.create_subject(oname=gname, gname=gname, _type=BoxObject, hexahedral=True)

# ## add blocking structure on table
# gscene.create_safe(gtype=GEOTYPE.BOX, name="blocker", link_name="base_link",
#                             center=(-0.27,-0.2,0.15), dims=(0.1,0.3, 0.4), rpy=(0,0,0), color=(0.0,0.8,0.0,0.5),
#                             collision=True, fixed=True)

### planners

In [11]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene, enable_dual=False)
mplan.update_gscene()
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

In [12]:
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.filtering.latticized_filter import LatticedChecker
from pkg.planning.filtering.task_clearance_filter import TaskClearanceChecker

gcheck = GraspChecker(pscene)
rcheck = ReachChecker(pscene)
tcheck = TaskClearanceChecker(pscene, gcheck)
checkers_all = [tcheck, rcheck, gcheck]
# lcheck = LatticedChecker(pscene, gcheck)
# checkers_all.append(lcheck)
mplan.motion_filters = checkers_all
gscene.show_pose(crob.home_pose)

from pkg.planning.constraint.constraint_common \
            import sample_redundancy, combine_redundancy
gtimer = GlobalTimer.instance()
initial_state = pscene.initialize_state(crob.home_pose, force_fit_binding=True)

# remove place points except for the current one
use_current_place_point_only(pscene, initial_state)

gscene.update_markers_all()
print("subjects: {}".format(pscene.subject_name_list))
print("initial node: {}".format(initial_state.node))

subjects: ['box1', 'box3', 'sweep1', 'sweep2', 'sweep3']
initial node: ('track_face', 'track_face', 0, 0, 0)


# CustomRule

In [13]:
from pkg.planning.task.custom_rules.sweep_entrance_control import SweepEntranceControlRule
from pkg.planning.constraint.constraint_common import get_binding_margins, fit_binding
from pkg.utils.traj_utils import simplify_schedule, mix_schedule
tplan.custom_rule = SweepEntranceControlRule(pscene)


# ## sampler
# from pkg.planning.sampling.node_sampling import make_state_param_hashable, UniformNodeSampler, PenaltyNodeSampler, GrowingSampler

# tplan.new_node_sampler = PenaltyNodeSampler(3, 1)
# tplan.parent_node_sampler = UniformNodeSampler(3)
# # tplan.parent_snode_sampler = GrowingSampler(3)

## Detect once

In [14]:
mplan.reset_log(False)
gtimer.reset()

with gtimer.block("detect"):
    for sname in pscene.subject_name_list:
        if isinstance(pscene.subject_dict[sname], AbstractObject):
            pscene.remove_subject(sname)
            
    gscene.clear_non_fixed()
    
    ## detect again
    gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])

    ## create boxes
    box_list =  [] 
    for gname in sorted(gtem_dict.keys()):
        box_new = pscene.create_subject(oname=gname, gname=gname, _type=BoxObject, hexahedral=True)
        box_new.register_binders(pscene, PlacePlane)
        box_list.append(box_new)
    
    initial_state = pscene.initialize_state(crob.home_pose, force_fit_binding=True)
    gscene.update_markers_all()

    # remove place points and sub-binders except for the current ones
    use_current_place_point_only(pscene, initial_state)
    use_current_sub_binders_only(pscene, initial_state)
    tplan.prepare()
    mplan.update_gscene()

## Plan & execute

In [15]:
from pkg.planning.motion.moveit.moveit_py import PlannerConfig
mplan.incremental_constraint_motion = True
config_dict =crob.get_robot_config_dict()
no_sdk_dict = {rname: "no_sdk" in config_dict[rname].specs and config_dict[rname].specs["no_sdk"] for rname in crob.robot_names}

In [16]:
from pkg.planning.constraint.constraint_common import get_binding_margins, fit_binding
crob.reset_connection(True,True)

indy0 = crob.robot_dict["indy0"]
indy1 = crob.robot_dict["indy1"]

indy1.grasp_ref = False
indy1.traj_vel = 5
with indy1:
    indy1.set_joint_vel_level(3)

with indy0:
    indy0.reset()
    
with indy1:
    indy1.reset()
    
time.sleep(3)

with indy0:
    while (indy0.get_robot_status()['resetting']):
        time.sleep(0.5)
    
with indy1:
    while (indy1.get_robot_status()['resetting']):
        time.sleep(0.5)

connection command:
indy0: True
indy1: True
Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.106)
Connect: Server IP (192.168.0.106)
Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.106)
Connect: Server IP (192.168.0.106)
Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.106)


### Test gripper

In [17]:
crob.grasp(True, True)
time.sleep(0.5)
crob.grasp(False, False)

Connect: Server IP (192.168.0.106)
Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.106)
Connect: Server IP (192.168.0.63)


### Test motion

In [18]:
crob.joint_move_make_sure(crob.home_pose+0.05)

Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.106)


In [19]:
crob.joint_move_make_sure(crob.home_pose)

Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.106)


In [28]:
mplan.reset_log(False)
gtimer.reset()

with gtimer.block("detect"):
    for sname in pscene.subject_name_list:
        if isinstance(pscene.subject_dict[sname], AbstractObject):
            pscene.remove_subject(sname)

    gscene.clear_non_fixed()

    ## detect again
    gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])

    ## create boxes
    box_list =  [] 
    for gname in sorted(gtem_dict.keys()):
        box_new = pscene.create_subject(oname=gname, gname=gname, _type=BoxObject, 
                                        hexahedral=True)
        box_new.register_binders(pscene, PlacePlane)
        box_list.append(box_new)

    initial_state = pscene.initialize_state(crob.home_pose, force_fit_binding=True)
    gscene.update_markers_all()

    # remove place points and sub-binders except for the current ones
    use_current_place_point_only(pscene, initial_state)
    use_current_sub_binders_only(pscene, initial_state)
    tplan.prepare()
    mplan.update_gscene()

print(initial_state.node)

gtimer.tic("firstmove")
obj_num = len(gtem_dict)
sweep_num = len(sweep_list)
from_state = initial_state
t_exe = None
snode_schedule_all = []
for sweep_idx in range(sweep_num):
    gcheck.put_banned = [track_list[sweep_idx][2]]
    sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(sweep_num)])
#     sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(2)])+(0,)
    goal_nodes = [("track_face",)*obj_num+sweep_goal]
    if sweep_idx < sweep_num-1:
        for i_s in range(obj_num):
            obj_goal = ["track_face"]*obj_num
            obj_goal[i_s] = "grip1"
            goal_nodes += [tuple(obj_goal)+sweep_goal]
    gtimer.tic("plan{}".format(sweep_idx))
    ppline.search(from_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, 
                  timeout_loop=5, multiprocess=True, timeout=0.5, timeout_constrained=2, add_homing=False, post_optimize=False,
                  terminate_on_first=False)
    gtimer.toc("plan{}".format(sweep_idx))
    snode_schedule = ppline.tplan.get_best_schedule(at_home=False)
    snode_schedule += ppline.add_return_motion(snode_schedule[-1], initial_state=None, timeout=0.5, try_count=3)
#     snode_schedule_wps = simplify_schedule(pscene, snode_schedule_ori, step_fractions=[0,1])
#     snode_schedule_safe = bspline_simple_schedule(pscene, snode_schedule_wps, 0.5, 1)
    snode_schedule_simple = simplify_schedule(pscene, snode_schedule)
#     snode_schedule_safe = calculate_safe_schedule(pscene, snode_schedule_simple, 0.5, 1)
    snode_schedule_safe = []
    for snode in snode_schedule_simple:
        if snode.traj is None:
            snode_schedule_safe.append(snode)
            continue
        else:
            robots_in_act = crob.get_robots_in_act(snode.traj, skip_not_connected=False)
            if all(map(lambda namepair: no_sdk_dict[namepair[0]], robots_in_act)):
                snode_schedule_safe.append(snode)
                continue
            else:
                snode_schedule_safe += calculate_safe_schedule(pscene, [snode], 1.0, 1.5)
#     double_sweep_motions(snode_schedule_safe)
#     snode_schedule = snode_schedule_safe
#     snode_schedule = mix_schedule(mplan, snode_schedule_safe)
    snode_schedule = snode_schedule_safe
    from_state = snode_schedule[-1].state
    if t_exe is not None:
        
        t_exe.join()
        assert ppline.execute_res, "Previous execution failed"
    else:
        gtimer.toc("firstmove")
#     ppline.execute_schedule(snode_schedule,auto_stop=False, 
#     mode_switcher=ModeSwitcher(pscene), 
#                            multiproc=True)
    t_exe = Thread(target=ppline.execute_schedule, args = (snode_schedule,), 
                    kwargs=dict(
                        auto_stop=False, mode_switcher=ModeSwitcher(pscene),
                        multiproc=False, error_stop_deg=None, one_by_one=True))
    t_exe.daemon = True
    t_exe.start()
    snode_schedule_all.append(snode_schedule)

if t_exe is not None:
    t_exe.join()

if ppline.execute_res:
    crob.stop_tracking()
    crob.joint_move_make_sure(crob.home_pose)
# else:
#     break

('track_face', 'track_face', 0, 0, 0)
Use 36/36 agents
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0)
Motion Filer Failure: TaskClearanceChecker
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0) = fail
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
Motion Filer Failure: ReachChecker
Motion Filer Failure: ReachChecker
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0) = fail
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0) = fail
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
try transition motion
try: 0 - ('track_face', 'track_face', 0, 

try transition motion
Motion Filer Failure: GraspChecker
result: 4 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'grip1', 1, 0, 0) = fail
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0) = fail
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0) = fail
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
try: 5 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0)
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0) = fail
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
try: 5 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0)
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0) = fail
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 

Motion Filer Failure: GraspChecker
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
try: 7 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0)
result: 9 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'grip1', 1, 0, 0) = fail
result: 8 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0) = fail
result: 7 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'grip1', 1, 0, 0) = fail
Motion Filer Failure: ReachChecker
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
Motion Filer Failure: ReachChecker
try: 10 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0)
result: 5 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0) = success
Motion Filer Failure: GraspChecker
result: 5 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0) = fail
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1

try transition motion
result: 13 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0) = fail
transition motion tried: True
try transition motion
try: 8 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
try joint motion
try: 12 - ('grip1', 'track_face', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0)
transition motion tried: True
branching: 2->15 (1.82/5.0 s, steps/err: 15(239.350795746 ms)/0.000940015552856)
try: 15 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0)
Motion Filer Failure: GraspChecker
Motion Filer Failure: GraspChecker
try: 8 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
Motion Filer Failure: TaskClearanceChecker
Motion Filer Failure: ReachChecker
try joint motion
try: 1 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0)
try transition motion
result: 1 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0) = success
result: 

try: 26 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'grip1', 1, 0, 0)
joint motion tried: True
try: 23 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0)
try: 25 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'grip1', 1, 0, 0)
try: 24 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'grip1', 1, 0, 0)
transition motion tried: True
try joint motion
result: 19 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'grip1', 1, 0, 0) = fail
Motion Filer Failure: TaskClearanceChecker
try: 23 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0)
try: 27 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0)
try: 28 - ('grip1', 'track_face', 0, 0, 0)->('grip1', 'track_face', 1, 0, 0)
result: 15 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0) = success
try joint motion
Motion Filer Failure: TaskClearanceChecker
try joint motion
result: 21 - ('track_face', 'grip1', 0, 0, 0)->('tra

...

Motion Filer Failure: GraspChecker
branching: 106->116 (4.69/5.0 s, steps/err: 26(190.336227417 ms)/0.026088436516)
Motion Filer Failure: GraspChecker
result: 6 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'grip1', 2, 2, 0) = fail
result: 108 - ('track_face', 'grip1', 2, 2, 0)->('track_face', 'grip1', 2, 2, 0) = success
joint motion tried: True
result: 7 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'grip1', 2, 2, 0) = fail
result: 7 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'grip1', 2, 2, 0) = fail
result: 109 - ('track_face', 'grip1', 2, 2, 2)->('track_face', 'grip1', 2, 2, 2) = success
try: 118 - ('track_face', 'grip1', 2, 2, 0)->('track_face', 'grip1', 2, 2, 1)
branching: 108->118 (4.71/5.0 s, steps/err: 12(197.690010071 ms)/0.0274492137041)
try: 116 - ('track_face', 'grip1', 2, 2, 0)->('track_face', 'grip1', 2, 2, 1)
result: 111 - ('track_face', 'grip1', 2, 2, 2)->('track_face', 'grip1', 2, 2, 2) = success
result: 40 - ('track_face', 'track_face'

try joint motion
try: 128 - ('track_face', 'grip1', 2, 2, 2)->('track_face', 'grip1', 2, 2, 2)
try: 127 - ('track_face', 'grip1', 2, 2, 2)->('track_face', 'grip1', 2, 2, 2)
ERROR sampling parent - NO SAMPLE REMAINED! Re-Start with initial state
result: 91 - ('track_face', 'grip1', 2, 2, 1)->('track_face', 'grip1', 2, 2, 2) = success
result: 96 - ('track_face', 'grip1', 2, 2, 1)->('track_face', 'grip1', 2, 2, 2) = success
Motion Filer Failure: GraspChecker
ERROR sampling parent - NO SAMPLE REMAINED! Re-Start with initial state
try: 126 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0)
try: 125 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 1)
try: 126 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 1)
branching: 45->126 (4.9/5.0 s, steps/err: 91(870.748996735 ms)/0.00158104655877)
try: 0 - ('grip1', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0)
try: 0 - ('grip1', 'track_face', 2, 2, 0)-

branching: 99->131 (5.07/5.0 s, steps/err: 112(764.620065689 ms)/0.030480698063)
end
result: 98 - ('track_face', 'grip1', 2, 2, 1)->('track_face', 'grip1', 2, 2, 2) = success
branching: 98->134 (5.08/5.0 s, steps/err: 45(843.374013901 ms)/0.00171941281113)
result: 130 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 1) = fail
transition motion tried: True
======================= terminated 27: max iteration time reached (1633793691/1633793686.84 s) ===============================
======================= terminated 21: max iteration time reached (1633793691/1633793686.84 s) ===============================
======================= terminated 14: max iteration time reached (1633793691/1633793686.84 s) ===============================
======================= terminated 18: max iteration time reached (1633793691/1633793686.84 s) ===============================
transition motion tried: True
======================= terminated 16: max iteration time reached (1633793691

======================= terminated 32: max iteration time reached (1633793692/1633793686.84 s) ===============================
======================= terminated 28: max iteration time reached (1633793692/1633793686.84 s) ===============================
joint motion tried: True
result: 114 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0) = success
result: 112 - ('track_face', 'grip1', 2, 2, 1)->('track_face', 'grip1', 2, 2, 2) = success
branching: 114->145 (5.22/5.0 s, steps/err: 33(641.068935394 ms)/0.0358072163071)
======================= terminated 19: max iteration time reached (1633793692/1633793686.84 s) ===============================
branching: 112->144 (5.22/5.0 s, steps/err: 45(684.560060501 ms)/0.00151956888895)
======================= terminated 24: max iteration time reached (1633793692/1633793686.84 s) ===============================
======================= terminated 20: max iteration time reached (1633793692/1633793686.84 s) ===============

======================= terminated 3: max iteration time reached (1633793692/1633793686.84 s) ===============================
joint motion tried: False
Motion Plan Failure
result: 130 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0) = fail
======================= terminated 5: max iteration time reached (1633793692/1633793686.84 s) ===============================
Connect: Server IP (192.168.0.106)
http://192.168.0.63:9990/param_setting?control_force0=0
Goal reached
Goal reached
binder: brush_face
rname: indy0
binder: grip1
rname: indy1
binder: None
binder: track_face
rname: None
binder: brush_face
rname: indy0
Connect: Server IP (192.168.0.106)
Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.106)
binder: brush_face
rname: indy0
binder: grip1
rname: indy1
binder: None
binder: track_face
rname: None
binder: brush_face
rname: indy0
Connect: Server IP (192.168.0.106)
Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.106)
Connect: 

In [20]:
crob.stop_tracking()
crob.joint_move_make_sure(crob.home_pose)

Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.63)


In [13]:
for snode_schedule in snode_schedule_all:
    ppline.play_schedule(snode_schedule, period=0.001)

('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0)
('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
('track_face', 'grip1', 0, 0, 0)->('track_face', 'grip1', 1, 0, 0)
('track_face', 'grip1', 1, 0, 0)->('track_face', 'grip1', 2, 0, 0)
('track_face', 'grip1', 2, 0, 0)->('track_face', 'grip1', 2, 0, 0)


In [27]:
indy1.traj_vel = 5
with indy1:
    indy1.set_joint_vel_level(3)

Connect: Server IP (192.168.0.106)


In [ ]:
raise(RuntimeError("Below is for testing - preventing auto run"))

## Plan & show

In [15]:
from pkg.planning.motion.moveit.moveit_py import PlannerConfig
mplan.incremental_constraint_motion = True
config_dict =crob.get_robot_config_dict()
no_sdk_dict = {rname: "no_sdk" in config_dict[rname].specs and config_dict[rname].specs["no_sdk"] for rname in crob.robot_names}

In [16]:
mplan.reset_log(False)
gtimer.reset()

with gtimer.block("detect"):
    for sname in pscene.subject_name_list:
        if isinstance(pscene.subject_dict[sname], AbstractObject):
            pscene.remove_subject(sname)
            
    gscene.clear_non_fixed()
    
    ## detect again
    gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])

    ## create boxes
    box_list =  [] 
    for gname in sorted(gtem_dict.keys()):
        box_new = pscene.create_subject(oname=gname, gname=gname, _type=BoxObject, hexahedral=True)
        box_new.register_binders(pscene, PlacePlane)
        box_list.append(box_new)
    
    initial_state = pscene.initialize_state(crob.home_pose, force_fit_binding=True)
    gscene.update_markers_all()

    # remove place points and sub-binders except for the current ones
    use_current_place_point_only(pscene, initial_state)
    use_current_sub_binders_only(pscene, initial_state)
    tplan.prepare()
    mplan.update_gscene()
    
print(initial_state.node)

gtimer.tic("firstmove")
obj_num = len(gtem_dict)
sweep_num = len(sweep_list)
from_state = initial_state
t_exe = None
snode_schedule_all = []
for sweep_idx in range(sweep_num):
    gcheck.put_banned = [track_list[sweep_idx][2]]
    sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(sweep_num)])
#     sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(2)])+(0,)
    goal_nodes = [("track_face",)*obj_num+sweep_goal]
    if sweep_idx < sweep_num-1:
        for i_s in range(obj_num):
            obj_goal = ["track_face"]*obj_num
            obj_goal[i_s] = "grip1"
            goal_nodes += [tuple(obj_goal)+sweep_goal]
    gtimer.tic("plan{}".format(sweep_idx))
    ppline.search(from_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, 
                  timeout_loop=5, multiprocess=True, timeout=0.5, timeout_constrained=2, add_homing=False, post_optimize=False,
                  terminate_on_first=False)
    gtimer.toc("plan{}".format(sweep_idx))
    snode_schedule = ppline.tplan.get_best_schedule(at_home=False)
    snode_schedule += ppline.add_return_motion(snode_schedule[-1], initial_state=None, timeout=0.5, try_count=3)
#     snode_schedule_wps = simplify_schedule(pscene, snode_schedule_ori, step_fractions=[0,1])
#     snode_schedule_safe = bspline_simple_schedule(pscene, snode_schedule_wps, 0.5, 1)
    snode_schedule_simple = simplify_schedule(pscene, snode_schedule)
#     snode_schedule_safe = calculate_safe_schedule(pscene, snode_schedule_simple, 0.5, 1)
    snode_schedule_safe = []
    for snode in snode_schedule_simple:
        if snode.traj is None:
            snode_schedule_safe.append(snode)
            continue
        else:
            robots_in_act = crob.get_robots_in_act(snode.traj, skip_not_connected=False)
            if all(map(lambda namepair: no_sdk_dict[namepair[0]], robots_in_act)):
                snode_schedule_safe.append(snode)
                continue
            else:
                snode_schedule_safe += calculate_safe_schedule(pscene, [snode], 0.5, 1)
#     double_sweep_motions(snode_schedule_safe)
#     snode_schedule = snode_schedule_safe
#     snode_schedule = mix_schedule(mplan, snode_schedule_safe)
    snode_schedule = snode_schedule_safe
    from_state = snode_schedule[-1].state
    if t_exe:
        t_exe.join()
    else:
        if len(snode_schedule_all)==0:
            gtimer.toc("firstmove")
    snode_schedule_all.append(snode_schedule)
#     t_exe = Thread(target=ppline.play_schedule, args = (snode_schedule,), kwargs=dict( period=0.01))
#     t_exe.start()
# t_exe.join()

('track_face', 'track_face', 0, 0, 0)
Use 36/36 agents
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0)
Motion Filer Failure: TaskClearanceChecker
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0) = fail
try transition motion
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
Motion Filer Failure: GraspChecker
Motion Filer Failure: GraspChecker
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0) = fail
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0) = fail
Motion Filer Failure: GraspChecker
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
try: 0

joint motion tried: True
result: 5 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'grip1', 1, 0, 0) = fail
Motion Filer Failure: GraspChecker
result: 4 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'grip1', 1, 0, 0) = fail
try: 1 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0)
try transition motion
Motion Filer Failure: GraspChecker
transition motion tried: True
Motion Filer Failure: GraspChecker
result: 1 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0) = fail
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0) = fail
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0) = fail
result: 1 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0) = fail
result: 3 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0) = success
Motion Filer Failure: ReachChecker
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0,

try transition motion
try: 1 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0)
result: 8 - ('grip1', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0) = success
Motion Filer Failure: GraspChecker
joint motion tried: True
Motion Filer Failure: GraspChecker
try transition motion
Motion Filer Failure: GraspChecker
result: 9 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0) = fail
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0) = fail
result: 9 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0) = fail
result: 9 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0) = success
try transition motion
transition motion tried: True
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0) = fail
try: 2 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0)
Motion Filer Failure: GraspChecker
branching: 8->11 (1.34/5.

result: 16 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'grip1', 1, 0, 0) = fail
try joint motion
try: 9 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
Motion Filer Failure: TaskClearanceChecker
branching: 3->19 (1.67/5.0 s, steps/err: 94(423.846006393 ms)/0.001569458061)
try: 7 - ('grip1', 'track_face', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0)
try: 19 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0)
Motion Filer Failure: GraspChecker
Motion Filer Failure: TaskClearanceChecker
Motion Filer Failure: GraspChecker
try: 20 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0)
Motion Filer Failure: ReachChecker
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0) = fail
try: 18 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
Motion Filer Failure: ReachChecker
Motion Filer Failure: TaskClearanceChecker
try: 2 - ('track_face', 'grip1', 0, 0, 0

Motion Filer Failure: ReachChecker
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
result: 7 - ('grip1', 'track_face', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0) = success
Motion Filer Failure: GraspChecker
result: 2 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0) = success
try: 17 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
Motion Filer Failure: ReachChecker
try: 7 - ('grip1', 'track_face', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0)
Motion Filer Failure: GraspChecker
result: 18 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0) = fail
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0) = success
branching: 2->24 (2.11/5.0 s, steps/err: 8(283.656835556 ms)/0.00176402105945)
try: 22 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
Motion Filer Failure: GraspChecker
Motion Filer Failure: ReachChecker
branching: 0->22

...

result: 0 - ('track_face', 'grip1', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0) = fail
try: 122 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0)
try constrained motion
try: 122 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 1)
result: 116 - ('grip1', 'track_face', 2, 2, 0)->('grip1', 'track_face', 2, 2, 1) = fail
try: 126 - ('grip1', 'track_face', 2, 2, 0)->('grip1', 'track_face', 2, 2, 1)
Motion Filer Failure: GraspChecker
result: 55 - ('track_face', 'track_face', 2, 2, 0)->('grip1', 'track_face', 2, 2, 0) = success
try: 127 - ('grip1', 'track_face', 2, 2, 0)->('grip1', 'track_face', 2, 2, 0)
try: 126 - ('grip1', 'track_face', 2, 2, 0)->('grip1', 'track_face', 2, 2, 0)
Motion Filer Failure: TaskClearanceChecker
result: 12 - ('track_face', 'track_face', 2, 2, 0)->('grip1', 'track_face', 2, 2, 0) = fail
result: 39 - ('grip1', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0) = fail
try: 125 - ('track_face', 'trac

result: 131 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 1) = fail
end
Motion Filer Failure: GraspChecker
result: 125 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 1) = success
joint motion tried: True
Motion Plan Failure
branching: 0->135 (4.93/5.0 s, steps/err: 15(192.810058594 ms)/0.00187480794415)
try: 102 - ('track_face', 'track_face', 2, 2, 0)->('grip1', 'track_face', 2, 2, 0)
constrained motion tried: True
transition motion tried: True
result: 13 - ('track_face', 'track_face', 2, 2, 0)->('grip1', 'track_face', 2, 2, 0) = fail
result: 99 - ('grip1', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0) = success
result: 4 - ('track_face', 'track_face', 2, 2, 0)->('grip1', 'track_face', 2, 2, 0) = fail
result: 113 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0) = success
try: 137 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0)
branching: 99->137 (4.96/

branching: 122->146 (5.08/5.0 s, steps/err: 22(358.440160751 ms)/0.0243292501755)
transition motion tried: True
result: 126 - ('grip1', 'track_face', 2, 2, 0)->('grip1', 'track_face', 2, 2, 0) = success
joint motion tried: True
result: 130 - ('grip1', 'track_face', 2, 2, 0)->('grip1', 'track_face', 2, 2, 1) = success
branching: 126->145 (5.1/5.0 s, steps/err: 68(330.399036407 ms)/0.0167019727034)
======================= terminated 18: max iteration time reached (1633792055/1633792050.9 s) ===============================
end
result: 115 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0) = success
result: 121 - ('grip1', 'track_face', 2, 2, 0)->('grip1', 'track_face', 2, 2, 1) = success
constrained motion tried: True
branching: 130->147 (5.11/5.0 s, steps/err: 30(254.748821259 ms)/0.00127505637617)
end
branching: 121->149 (5.12/5.0 s, steps/err: 62(406.740903854 ms)/0.00149234616573)
======================= terminated 31: max iteration time reached (1633792056

======================= terminated 23: max iteration time reached (1633792056/1633792050.9 s) ===============================
result: 141 - ('grip1', 'track_face', 2, 2, 0)->('grip1', 'track_face', 2, 2, 0) = success
end
======================= terminated 13: max iteration time reached (1633792056/1633792050.9 s) ===============================
branching: 141->157 (5.21/5.0 s, steps/err: 17(203.132152557 ms)/0.0301578944875)
end
======================= terminated 29: max iteration time reached (1633792056/1633792050.9 s) ===============================
constrained motion tried: True
constrained motion tried: True
end
======================= terminated 10: max iteration time reached (1633792056/1633792050.9 s) ===============================
constrained motion tried: True
end
Goal reached
result: 100 - ('grip1', 'track_face', 2, 2, 1)->('grip1', 'track_face', 2, 2, 2) = success
result: 111 - ('grip1', 'track_face', 2, 2, 1)->('grip1', 'track_face', 2, 2, 2) = success
branching: 111->158

end
constrained motion tried: True
Goal reached
result: 136 - ('track_face', 'track_face', 2, 2, 1)->('track_face', 'track_face', 2, 2, 2) = success
branching: 136->167 (5.59/5.0 s, steps/err: 45(615.456819534 ms)/0.0015441263446)
======================= terminated 9: max iteration time reached (1633792056/1633792050.9 s) ===============================
transition motion tried: False
Motion Plan Failure
result: 42 - ('track_face', 'track_face', 2, 2, 0)->('grip1', 'track_face', 2, 2, 0) = fail
======================= terminated 2: max iteration time reached (1633792056/1633792050.9 s) ===============================
Goal reached
Goal reached


In [17]:
print(gtimer)

detect: 	299.3 ms/1 = 299.3 ms (299.263/299.263)
firstmove: 	6535.3 ms/1 = 6535.3 ms (6535.27/6535.27)
plan0: 	5673.7 ms/1 = 5673.7 ms (5673.672/5673.672)
initialize_memory: 	52.4 ms/3 = 17.5 ms (14.716/21.489)
init_search: 	18.6 ms/3 = 6.2 ms (3.458/8.18)
start_process: 	8076.8 ms/3 = 2692.3 ms (2510.018/2794.503)
wait_procs: 	22686.4 ms/15 = 1512.4 ms (501.461/2536.059)
wait_procs_extra: 	317951.4 ms/252 = 1261.7 ms (0.003/2769.676)
plan1: 	8100.4 ms/1 = 8100.4 ms (8100.378/8100.378)
plan2: 	5813.3 ms/1 = 5813.3 ms (5813.298/5813.298)



In [18]:
# for i_ss, snode_schedule in enumerate(snode_schedule_all):
#     for i_s, snode in enumerate(snode_schedule):
#         if snode.traj is not None:
#             save_json("data/traj_{}_{}.json".format(i_ss, i_s), snode.traj[:,:6])

## mix full schedule

In [18]:
gtimer.reset()
snode_schedule_cat = [snode_schedule_all[0][0]]
for snode_schedule in snode_schedule_all:
    snode_schedule_cat += snode_schedule[1:]

# with gtimer.block("mix_schedule_safe"):
#     safe_mixed = mix_schedule(mplan, snode_schedule_cat)

# print(gtimer)

safe_mixed = snode_schedule_cat

### play schedule

In [19]:
ppline.play_schedule(safe_mixed, period=0.001)

('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
('track_face', 'grip1', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0)
('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
('grip1', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
('grip1', 'track_face', 0, 0, 0)->('grip1', 'track_face', 1, 0, 0)
('grip1', 'track_face', 1, 0, 0)->('grip1', 'track_face', 2, 0, 0)
('grip1', 'track_face', 2, 0, 0)->('grip1', 'track_face', 2, 0, 0)
('grip1', 'track_face', 2, 0, 0)->('grip1', 'track_face', 2, 0, 0)
('grip1', 'track_face', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0)
('track_face', 'track_face', 2, 0, 0)->('track_face', 'grip1', 2, 0, 0)
('track_face', 'grip1', 2, 0, 0)->('track_face', 'grip1', 2, 0, 0)
('track_face', 'grip1', 2, 0, 0)->('track_face', 'grip1', 2, 1, 0)
('track_face', 'grip1', 2, 1, 0)->('track_face', 'grip1', 2, 2, 0)
('track_face', 'grip1', 2, 2, 0)->('track_face', 'grip1', 2, 2, 0)
('track_face', 'grip1', 2, 2, 0)->('t

In [21]:
# ppline.play_schedule(safe_mixed[:1], period=0.001)
# color_off = (0.8,0.2,0.2,0.2)
# for obj_name in ppline.pscene.subject_name_list:
#     if isinstance(ppline.pscene.subject_dict[obj_name], AbstractTask):
#         actor, obj = ppline.pscene.actor_dict['brush_face'], ppline.pscene.subject_dict[obj_name]
#         for gtem in obj.clearance:
#             gtem.color = color_off
#             ppline.pscene.gscene.update_marker(gtem)
#             ppline.pscene.set_object_state(snode_schedule_all[0][0].state)
# ppline.pscene.gscene.update_markers_all()

In [22]:
# from pkg.utils.code_scraps import play_schedule_clearance_highlight
# play_schedule_clearance_highlight(ppline, safe_mixed, tcheck=tcheck, period=0.001)

### Run fully mixed schedule

In [20]:
from pkg.planning.constraint.constraint_common import get_binding_margins, fit_binding
crob.reset_connection(True,True)

connection command:
indy0: True
indy1: True
Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.106)


In [21]:
crob.stop_tracking()

Connect: Server IP (192.168.0.63)


In [22]:
indy0 = crob.robot_dict["indy0"]
indy1 = crob.robot_dict["indy1"]

indy1.grasp_ref = False
with indy1:
    indy1.set_joint_vel_level(1)

with indy0:
    indy0.reset()
    
with indy1:
    indy1.reset()
    
time.sleep(3)

Connect: Server IP (192.168.0.106)
Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.106)
Connect: Server IP (192.168.0.106)


In [23]:
crob.grasp(True, True)
time.sleep(0.5)
crob.grasp(False, False)

Connect: Server IP (192.168.0.106)
Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.106)
Connect: Server IP (192.168.0.63)


In [24]:
crob.joint_move_make_sure(crob.home_pose+0.05)

Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.106)


In [25]:
crob.joint_move_make_sure(crob.home_pose)

Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.106)


In [26]:
ppline.execute_schedule(safe_mixed, mode_switcher=ModeSwitcher(pscene),
                        multiproc=False, error_stop_deg=None, one_by_one=True)

Connect: Server IP (192.168.0.106)
Connect: Server IP (192.168.0.106)
Connect: Server IP (192.168.0.106)
Move finished!
Connect: Server IP (192.168.0.106)
Connect: Server IP (192.168.0.106)
binder: None
binder: track_face
rname: None
binder: None
binder: grip1
rname: indy1
binder: None
Connect: Server IP (192.168.0.106)
Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.106)
Connect: Server IP (192.168.0.106)
Connect: Server IP (192.168.0.106)
Move finished!
Connect: Server IP (192.168.0.106)
Connect: Server IP (192.168.0.106)
binder: None
binder: track_face
rname: None
binder: None
binder: track_face
rname: None
binder: None
Connect: Server IP (192.168.0.106)
Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.106)
Connect: Server IP (192.168.0.106)
Connect: Server IP (192.168.0.106)
Move finished!
Connect: Server IP (192.168.0.106)
Connect: Server IP (192.168.0.106)
binder: None
binder: grip1
rname: indy1
binder: None
binder: track_face
rname: None
binder: No

True